In [2]:
## DATA CLEANING 

In [3]:
import numpy as np
import pandas as pd

In [4]:
## Loading The Insurance claim dataset
df = pd.read_csv(r"C:\Users\PAUL\Desktop\ML_Projects\dataset\Train_data.csv")

In [5]:
# Count missing values in each column
missing_count = df.isnull().sum()
missing_percent = (missing_count / len(df)) * 100

print("Missing values (count):")
print(missing_count.sort_values(ascending=False))
print("\nMissing values (percentage):")
print(missing_percent.sort_values(ascending=False))

Missing values (count):
Date_of_Occupancy     508
Building Dimension    106
Geo_Code              102
Garden                  7
Customer Id             0
YearOfObservation       0
Building_Fenced         0
Building_Painted        0
Residential             0
Insured_Period          0
Building_Type           0
Settlement              0
NumberOfWindows         0
Claim                   0
dtype: int64

Missing values (percentage):
Date_of_Occupancy     7.094972
Building Dimension    1.480447
Geo_Code              1.424581
Garden                0.097765
Customer Id           0.000000
YearOfObservation     0.000000
Building_Fenced       0.000000
Building_Painted      0.000000
Residential           0.000000
Insured_Period        0.000000
Building_Type         0.000000
Settlement            0.000000
NumberOfWindows       0.000000
Claim                 0.000000
dtype: float64


In [6]:
## Checking for duplicated records

duplicate_customer_ids = df['Customer Id'].duplicated().sum()

In [7]:
# List of categorical columns to examine
categorical_cols = ['Building_Painted', 'Building_Fenced', 'Garden', 'Settlement', 'NumberOfWindows']

# Show unique values and their counts for each column
for col in categorical_cols:
    print(f"\n{col}:")
    print(df[col].value_counts())


Building_Painted:
Building_Painted
V    5382
N    1778
Name: count, dtype: int64

Building_Fenced:
Building_Fenced
N    3608
V    3552
Name: count, dtype: int64

Garden:
Garden
O    3602
V    3551
Name: count, dtype: int64

Settlement:
Settlement
R    3610
U    3550
Name: count, dtype: int64

NumberOfWindows:
NumberOfWindows
   .    3551
4        939
3        844
5        639
2        363
6        306
7        211
8        116
1         75
>=10      67
9         49
Name: count, dtype: int64


Data cleaning workflow to be used:
1. Convert data types (e.g., text to numbers)
2. Standardize categorical values (consistent formatting)
3. Handle missing values (imputation)
4. Remove duplicates
5. Verify final data quality

In [8]:
# Creating a copy for cleaning - preserves original data

df_clean = df.copy()

print("Starting data cleaning...")
print(f"Original shape: {df_clean.shape}")

Starting data cleaning...
Original shape: (7160, 14)


In [9]:
# Convert NumberOfWindows from text to number
# Invalid values will become NaN (we'll fill them later)
df_clean['NumberOfWindows'] = pd.to_numeric(df_clean['NumberOfWindows'], errors='coerce')
print(f"NumberOfWindows converted to numeric")
print(f"Missing values after conversion: {df_clean['NumberOfWindows'].isnull().sum()}")

NumberOfWindows converted to numeric
Missing values after conversion: 3618


In [10]:
df_clean.info()  ## Confirmed

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7160 entries, 0 to 7159
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Customer Id         7160 non-null   object 
 1   YearOfObservation   7160 non-null   int64  
 2   Insured_Period      7160 non-null   float64
 3   Residential         7160 non-null   int64  
 4   Building_Painted    7160 non-null   object 
 5   Building_Fenced     7160 non-null   object 
 6   Garden              7153 non-null   object 
 7   Settlement          7160 non-null   object 
 8   Building Dimension  7054 non-null   float64
 9   Building_Type       7160 non-null   int64  
 10  Date_of_Occupancy   6652 non-null   float64
 11  NumberOfWindows     3542 non-null   float64
 12  Geo_Code            7058 non-null   object 
 13  Claim               7160 non-null   int64  
dtypes: float64(4), int64(4), object(6)
memory usage: 783.3+ KB


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7160 entries, 0 to 7159
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Customer Id         7160 non-null   object 
 1   YearOfObservation   7160 non-null   int64  
 2   Insured_Period      7160 non-null   float64
 3   Residential         7160 non-null   int64  
 4   Building_Painted    7160 non-null   object 
 5   Building_Fenced     7160 non-null   object 
 6   Garden              7153 non-null   object 
 7   Settlement          7160 non-null   object 
 8   Building Dimension  7054 non-null   float64
 9   Building_Type       7160 non-null   int64  
 10  Date_of_Occupancy   6652 non-null   float64
 11  NumberOfWindows     7160 non-null   object 
 12  Geo_Code            7058 non-null   object 
 13  Claim               7160 non-null   int64  
dtypes: float64(3), int64(4), object(7)
memory usage: 783.3+ KB


In [12]:
# List of categorical columns to clean
categorical_cols = ['Building_Painted', 'Building_Fenced', 'Garden', 'Settlement']

# Clean each categorical column
for col in categorical_cols:
    # Remove spaces and convert to Title Case (Yes, No, etc.)
    df_clean[col] = df_clean[col].astype(str).str.strip().str.title()
    
    # Replace string "Nan" with actual NaN (missing value)
    df_clean[col] = df_clean[col].replace('Nan', np.nan)
    df_clean[col] = df_clean[col].replace('nan', np.nan)
    
    print(f"{col} cleaned")

Building_Painted cleaned
Building_Fenced cleaned
Garden cleaned
Settlement cleaned


In [13]:
"""
CLEANING STEP 3: IMPUTE MISSING VALUES (MISSING DATA HANDLING)
================================================================
Purpose: Fill in missing values using appropriate statistical methods.
"""

"""
Imputation Strategy by Column:
==============================

1. Garden (Categorical, 0.1% missing):
   - Strategy: Mode (most common value)
   - Reason: Small percentage, mode preserves category distribution
   
2. Building Dimension (Numerical, 1.48% missing):
   - Strategy: Median
   - Reason: Median is robust to outliers (unlike mean)
   - Preserves central tendency without skewing from extreme values
   
3. Date_of_Occupancy (Numerical, 7.09% missing):
   - Strategy: Median
   - Reason: Dates can have outliers, median provides stable estimate
   
4. Geo_Code (Categorical, 1.42% missing):
   - Strategy: Mode (most common geographic code)
   - Reason: Categorical data best represented by most frequent category
   
5. NumberOfWindows (Numerical, if any missing):
   - Strategy: Median
   - Reason: Window counts are discrete numbers, median is appropriate 
"""

# Fill missing values in Garden (categorical) with most common value
garden_mode = df_clean['Garden'].mode()[0]
df_clean['Garden'].fillna(garden_mode, inplace=True)
print(f"Garden: filled with '{garden_mode}'")

# Fill missing values in Building Dimension (numerical) with median
building_dim_median = df_clean['Building Dimension'].median()
df_clean['Building Dimension'].fillna(building_dim_median, inplace=True)
print(f"Building Dimension: filled with median {building_dim_median:.2f}")

# Fill missing values in Date_of_Occupancy (numerical) with median
occupancy_median = df_clean['Date_of_Occupancy'].median()
df_clean['Date_of_Occupancy'].fillna(occupancy_median, inplace=True)
print(f"Date_of_Occupancy: filled with median {occupancy_median:.2f}")

# Fill missing values in Geo_Code (categorical) with most common value
geo_mode = df_clean['Geo_Code'].mode()[0]
df_clean['Geo_Code'].fillna(geo_mode, inplace=True)
print(f"Geo_Code: filled with '{geo_mode}'")

# Fill missing values in NumberOfWindows (numerical) with median (if any)
if df_clean['NumberOfWindows'].isnull().sum() > 0:
    windows_median = df_clean['NumberOfWindows'].median()
    df_clean['NumberOfWindows'].fillna(windows_median, inplace=True)
    print(f"NumberOfWindows: filled with median {windows_median:.2f}")

Garden: filled with 'O'


Building Dimension: filled with median 1083.00
Date_of_Occupancy: filled with median 1970.00
Geo_Code: filled with '6088'
NumberOfWindows: filled with median 4.00


C:\Users\PAUL\AppData\Local\Temp\ipykernel_10032\660543993.py:35: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_clean['Garden'].fillna(garden_mode, inplace=True)
C:\Users\PAUL\AppData\Local\Temp\ipykernel_10032\660543993.py:40: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For e

In [14]:
"""
CLEANING STEP 4: REMOVE DUPLICATE ROWS
=======================================
Purpose: Eliminate completely identical records from the dataset""""Data Cleaning.ipynb"

# Count duplicates before removal
duplicates_count = df_clean.duplicated().sum()

# Remove duplicate rows
df_clean = df_clean.drop_duplicates()

print(f"Removed {duplicates_count} duplicate rows")


Removed 0 duplicate rows


In [15]:
### Confirm that all missing values have been successfully imputed using If Else 

remaining_missing = df_clean.isnull().sum()

print("\nRemaining missing values after cleaning:")
if remaining_missing.sum() == 0:
    print("✓ No missing values remaining!")
else:
    print(remaining_missing[remaining_missing > 0])


Remaining missing values after cleaning:
✓ No missing values remaining!


In [16]:
## Final data type and shap
print("Final data types:")
print(df_clean.dtypes)
print(f"\nFinal shape: {df_clean.shape}")

Final data types:
Customer Id            object
YearOfObservation       int64
Insured_Period        float64
Residential             int64
Building_Painted       object
Building_Fenced        object
Garden                 object
Settlement             object
Building Dimension    float64
Building_Type           int64
Date_of_Occupancy     float64
NumberOfWindows       float64
Geo_Code               object
Claim                   int64
dtype: object

Final shape: (7160, 14)


In [17]:
## checking numerical columns for outliers

# List of numerical columns
numerical_cols = ['YearOfObservation', 'Insured_Period', 'Residential', 'Building Dimension', 
                  'Building_Type', 'Date_of_Occupancy', 'NumberOfWindows']

# Display summary statistics
print("Summary statistics for numerical columns:")
print(df_clean[numerical_cols].describe())

Summary statistics for numerical columns:
       YearOfObservation  Insured_Period  Residential  Building Dimension  \
count        7160.000000     7160.000000  7160.000000         7160.000000   
mean         2013.669553        0.909758     0.305447         1871.873184   
std             1.383769        0.239756     0.460629         2263.296186   
min          2012.000000        0.000000     0.000000            1.000000   
25%          2012.000000        0.997268     0.000000          531.500000   
50%          2013.000000        1.000000     0.000000         1083.000000   
75%          2015.000000        1.000000     1.000000         2250.000000   
max          2016.000000        1.000000     1.000000        20940.000000   

       Building_Type  Date_of_Occupancy  NumberOfWindows  
count    7160.000000        7160.000000      7160.000000  
mean        2.186034        1964.849721         4.111453  
std         0.940632          34.730347         1.165521  
min         1.000000        

In [18]:
print("=" * 50)
print("CLEANED DATASET SUMMARY")
print("=" * 50)
df_clean.info()
print("\n✓ Data cleaning completed successfully!")

CLEANED DATASET SUMMARY
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7160 entries, 0 to 7159
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Customer Id         7160 non-null   object 
 1   YearOfObservation   7160 non-null   int64  
 2   Insured_Period      7160 non-null   float64
 3   Residential         7160 non-null   int64  
 4   Building_Painted    7160 non-null   object 
 5   Building_Fenced     7160 non-null   object 
 6   Garden              7160 non-null   object 
 7   Settlement          7160 non-null   object 
 8   Building Dimension  7160 non-null   float64
 9   Building_Type       7160 non-null   int64  
 10  Date_of_Occupancy   7160 non-null   float64
 11  NumberOfWindows     7160 non-null   float64
 12  Geo_Code            7160 non-null   object 
 13  Claim               7160 non-null   int64  
dtypes: float64(4), int64(4), object(6)
memory usage: 783.3+ KB

✓ Data cleaning comp

In [19]:
df_clean.to_csv('clean_dataset.csv', index=False)